# DSCI 575 - Advanced Machine Learning

# Lab 1: Text generation using Markov Models

## Table of contents
- [Submission guidelines](#sg) (4%)
- [Exercise 1: Warm-up](#1) (20%)
- [Exercise 2: Character-based Markov model of language](#2) (20%)
- [Exercise 3: Stationary distribution and other fun stuff](#3) (30%)
- [Exercise 4: Word-based Markov model of language](#4) (26%)
- [Submission to Canvas](#sc)

## Submission instructions <a name="si"></a>
<hr>
rubric={mechanics:4}

You will receive marks for correctly submitting this assignment. 

To correctly submit this assignment follow the instructions below:

- Push your assignment to your GitHub repository. 
- Add a link to your GitHub repository here: LINK TO YOUR GITHUB REPO  https://github.ubc.ca/MDS-2020-21/DSCI_575_lab1_aish912
- Upload an HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).

[Here](https://github.com/UBC-MDS/public/tree/master/rubric) you will find the description of each rubric used in MDS.

**NOTE: The data you download for use in this lab SHOULD NOT BE PUSHED TO YOUR REPOSITORY. You might be penalised for pushing datasets to your repository. I have seeded the repository with `.gitignore` and hoping that it won't let you push CSVs.**

<br><br><br><br>

## Imports <a name="im"></a>

In [105]:
import os
import re
import sys
from collections import Counter, defaultdict
from urllib.request import urlopen

import numpy as np
import numpy.random as npr
import pandas as pd

<br><br><br><br>

## Exercise 1: Warm-up <a name="1"></a>

This exercise will get you thinking about how to generate text using Markov models. In the lecture, we built a Markov model of language on a toy corpus using words. In this exercise, you will build a letter or character-based Markov model of language with a tiny corpus. The goal of such a model is to predict next character given a sequence of characters. 

In NLP, a Markov model of language is also referred to as **an n-gram language model**. In this exercise we'll focus on a character-based bigram (2-gram) language model. We will use the variable `n=1` to denote that we are only considering the current character in the sequence to predict the next character. In the next exercise, you'll explore different values for `n` in your n-gram language model. 
 
As we saw in class, to build a bigram model, we need bigram frequency counts, i.e., counts of all 2-letter sequences in your corpus. Below I am providing you some starter code to create bigram frequency counts of letters in a tiny corpus with 6 words. Our goal is to build a character-based bigram language model with this corpus, where the set of states is the unique letters in the corpus.  

> When we say n-gram we're now referring to the last `n` characters, not the last `n` words. In general, the term n-gram can refer to characters or word; see the [Wikipedia article](https://en.wikipedia.org/wiki/N-gram).


> Recall that in [DSCI 512 lab1](https://github.ubc.ca/MDS-2020-21/DSCI_512_alg-data-struct_students/blob/master/solutions/lab1/lab1.ipynb), you implemented a Markov model of language and generated text (without actually knowing the details about the model). You pre-computed the frequencies for every possible `n` gram as follows.  

In [135]:
corpus = "to be or not to be"  # our tiny corpus
n = 1  # for bigrams
circ_corpus = corpus + corpus[:n]
frequencies = defaultdict(Counter)
for i in range(len(circ_corpus) - n):
    frequencies[circ_corpus[i : i + n]][circ_corpus[i + n]] += 1
frequencies

defaultdict(collections.Counter,
            {'t': Counter({'o': 2, ' ': 1}),
             'o': Counter({' ': 2, 'r': 1, 't': 1}),
             ' ': Counter({'b': 2, 'o': 1, 'n': 1, 't': 1}),
             'b': Counter({'e': 2}),
             'e': Counter({' ': 1, 't': 1}),
             'r': Counter({' ': 1}),
             'n': Counter({'o': 1})})

<br><br>

### 1.1 Visualizing character bigram counts as a co-occurrence matrix
rubric={accuracy:4} 

**Your tasks:**

1. Show the bigram frequencies in `frequencies` variable above as a pandas DataFrame, sorting the column and row labels alphabetically, where
    * column labels and row indices are unique characters in the corpus, and 
    * the value in each cell $a_{ij}$ represents how often the character $i$ precedes character $j$ in our corpus.
    
> Note: Fill in the NaN values with zeros. 

In [136]:
### Solution_1_1_1

### YOUR ANSWER HERE
frequencies_df = pd.DataFrame.from_dict(frequencies, orient = 'index')
frequencies_df.sort_index(axis=1, inplace=True)
frequencies_df.sort_index(axis=0, inplace=True)
frequencies_df = frequencies_df.replace(np.nan, 0)
frequencies_df

,,b,e,n,o,r,t
,0.0,2.0,0.0,1.0,1.0,0.0,1.0
b,0.0,0.0,2.0,0.0,0.0,0.0,0.0
e,1.0,0.0,0.0,0.0,0.0,0.0,1.0
n,0.0,0.0,0.0,0.0,1.0,0.0,0.0
o,2.0,0.0,0.0,0.0,0.0,1.0,1.0
r,1.0,0.0,0.0,0.0,0.0,0.0,0.0
t,1.0,0.0,0.0,0.0,2.0,0.0,0.0


<br><br>

### 1.2 Transition matrix
rubric={accuracy:6}


**Your tasks:**

1. Given the frequencies in 1.1, compute the transition matrix (i.e., the conditional probability distribution for every possible bigram). Visualize the transition matrix as a pandas DataFrame.  
2. How many unique states are there in your Markov model? 

> Recall that the transition matrix $T$ is a square matrix and the number of rows/columns is equal to the number of states. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

In [137]:
### Solution_1_2_1

### YOUR ANSWER HERE

for index, row in frequencies_df.iterrows():
    n = np.nansum(row)
    row[:] = [round(f/n,2) for f in row]
frequencies_df

,,b,e,n,o,r,t
,0.00,0.4,0.0,0.2,0.20,0.00,0.20
b,0.00,0.0,1.0,0.0,0.00,0.00,0.00
e,0.50,0.0,0.0,0.0,0.00,0.00,0.50
n,0.00,0.0,0.0,0.0,1.00,0.00,0.00
o,0.50,0.0,0.0,0.0,0.00,0.25,0.25
r,1.00,0.0,0.0,0.0,0.00,0.00,0.00
t,0.33,0.0,0.0,0.0,0.67,0.00,0.00


In [86]:
### YOUR ANSWER HERE

### Solution_1_2_2

### YOUR ANSWER HERE
There are seven unique states in the markov model.

<br><br>

### 1.3 Probability of sequences
rubric={reasoning:4}

**Your tasks:**

Suppose the probability of starting a sequence with letter $b$ is $0.4$ (i.e., $\pi_0$ of state $b$ = 0.4). Given the transition matrix in 1.2, calculate the probabilities for the following sequences. 

1. "be or"
2. "beet"

**Solution_1_3**
### YOUR ANSWER HERE

1. P(b) * P(e|b) * P( |e) * P(o| ) * P(r|o) 

   0.4 * 1.0 * 0.5 * 0.2 * 0.25 = 
   
  
2. P(b) * P(e|b) * P(e|e) * P(t|e) 

   0.4 * 1.0 * 0 * 0.5
   

<br><br>

### 1.4 Generate sequences
rubric={reasoning:6}

**Your tasks:**

1. Given the transition matrix above, explain in words how would you generate a sequence of length `seq_len` given the seed letter `t`. You don't have to, but you are welcome to write code for this. 
2. Give a few example applications for word- or character-level text generation. 

**Solution_1_4_1**

### YOUR ANSWER HERE

Given the first character is t, from the transition matrix, we can observe that, the following charcters can either be a space or the letter 'n'. Once the next letter is chosen from the distribution of " " and "n", that character becomes the current state. For instance, if n is chosen, then from the transition matrix, the possible character is only "o". This becomes the current state. This process continues until the length condition is satisfied.

In [6]:
### YOUR ANSWER HERE

**Solution_1_4_1**

### YOUR ANSWER HERE

An application of character-level text generation is the auto-complete in our text applications.

Application of word-level text generation is the gmail autocomplete feature or the dropdown autocomplete in youtube, google etc.

<br><br><br><br>

## Exercise 2: Character-based Markov model of language <a name="2"></a>

Recall Markov Model of language from [DSCI 512 lab1](https://github.ubc.ca/MDS-2020-21/DSCI_512_alg-data-struct_students/blob/master/solutions/lab1/lab1.ipynb). In that lab, you wrote a class `MarkovModel` to `fit` an n-gram Markov model and generated text. Now that you know what Markov models are, we will start with what you did in DSCI 512 lab1 and build on it. 

The starter code below uses the hyperparameter `n`, where our "state" of the Markov chain is the last `n` characters of a give string. In Exercise 1, we worked with `n=1` (bigram model) and our "state" of the Markov chain was a single character and each character was dependent on the last one character. When `n=3`, it means that the probability distribution over the _next character_ only depends on the _preceding 3 characters_. 

> Note that `n` in n-gram doesn't exactly correspond to the variable `n` we are using in the implementation below. For 2-gram (bigram) the value of the variable `n` is 1, and for 4-gram the value of `n=3`. 

We train our model from data by recording every occurrence of every n-gram and, in each case, recording what the next letter is. Then, for each n-gram, we normalize these counts into probabilities just like we did with naive Bayes. The `fit` function below implements these steps.

To generate a new sequence, we start with some initial seed at least of length `n` (here, we will just use the first `n` characters in the training text, which are saved at the end of the `fit` function). Then, for the current n-gram we will look up the probability distribution over next characters and sample a random character according to this distribution.

Attribution: assignment adapted with permission from Princeton COS 126, [_Markov Model of Natural Language_]( http://www.cs.princeton.edu/courses/archive/fall15/cos126/assignments/markov.html). Original assignment was developed by Bob Sedgewick and Kevin Wayne. If you are interested in more background info, you can take a look at the original version. The original paper by Shannon, [A Mathematical Theory of Communication](http://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf), essentially created the field of information theory and is one of the best scientific papers ever written (in terms of both impact and readability).  

In [140]:
class MarkovModel:
    def __init__(self, n):
        """
        Initialize the Markov model object.

        Parameters:
        ----------
        n : int
            the size of the ngram
        """
        self.n = n

    def fit(self, text):
        """
        Fit a Markov chain and create a transition matrix.

        Parameters
        ----------
        text : str
            text to fit the Markov chain
        """

        # make text circular so markov chain doesn't get stuck
        circ_text = text + text[: self.n]
#         circ_text = text
        # count the number of occurrences of each letter following a given n-gram
        frequencies = defaultdict(Counter)
        for i in range(len(text)):
            frequencies[circ_text[i : i + self.n]][circ_text[i + self.n]] += 1.0

        # normalize the frequencies into probabilities (separately for each n-gram)
        self.probabilities_ = defaultdict(dict)
        for ngram, counts in frequencies.items():
            self.probabilities_[ngram]["symbols"] = list(counts.keys())
            probs = np.array(list(counts.values()))
            probs /= np.sum(probs)
            self.probabilities_[ngram]["probs"] = probs

        # store the first n characters of the training text, as we will use these
        # to seed our `generate` function
        self.starting_chars = text[: self.n]
        print(len(self.probabilities_))
        self.frequencies_ = frequencies  # you never know when this might come in handy

    def generate(self, seq_len):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
        s = self.starting_chars
        while len(s) < seq_len:
            probs = self.probabilities_[s[-self.n :]]
            s += npr.choice(probs["symbols"], p=probs["probs"])
        return s

<br><br>

Let's test our code. 

In [141]:
# Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm
data_url = "http://www.gutenberg.org/files/2591/2591-0.txt"

corpus = urlopen(data_url).read().decode("utf-8")
corpus = corpus[2000:]
model = MarkovModel(n=2)
model.fit(corpus)
print(model.generate(500))

1557
E Prope, ‘You up

priall justleese bosece se evere cand ponce, at carrathe
as lig, so stablover eve
As anyth linge, and thew asithenintoods th moung crind heregifulew balred.

withat countop carfs; a plead
1.F.6. AND Tren and ther, she fet ong wher teriger, and don wast of it, the horrow,
you was and wearnin the abin evere afte, had him dwas pas you shed: liked be
is ang fir took doothe upon an to but mang.’ st might his st th of the my
what ree mand he stlerldis ands pluck tone kin th


<br><br>

### 2.1 "Circular" version of the text
rubric={reasoning:5}

**Your taks:**

Why do we need to use a "circular" version of the text in the `fit` function? What could go wrong if we didn't do that, and instead used the original `text` (instead of `circ_text`) and made the loop:  
```
for i in range(len(text)-self.n):
```
which allow `fit` to run without crashing?

> Hint: the problem would arise during `generate`, not during `fit`.



**Solution_2.1**

### YOUR ANSWER HERE

If we do not append the first n-gram at the end, then when we encounter the last n-gram in the generate function, there won't be any prediction letters for the last n-gram and the generate function will throw an error. 

<br><br>

### (Optional) 2.2 Smoothing
rubric={reasoning:1}

The description above mentioned a connection to naive Bayes. When discussing naive Bayes we briefly mentioned [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing). Is there something analogous to Laplace smoothing that we could do here? If so, describe what it would entail and what effect it would have. (You don't have to implement it.)

**Solution_2.2**
### YOUR ANSWER HERE

We could include all possible ngrams of length n, and add 1 to the count of all ngrams before normalizing. This will allow us to generate unseen patterns with very small but not zero probability.

<br><br>

### 2.3 States, state space, and transition matrix
rubric={reasoning:15}

Let's consider the Markov chain interpretation of what we've just done. Let's define a state as the previous $n$ characters "emitted" by our chain. Let's assume our vocabulary size (the number of possible characters) is $V$; for example, $V=26$ if we were only using lowercase letters. We can compute this in our case:

In [108]:
print("Vocabulary size = %d" % len(np.unique(list(corpus))))

Vocabulary size = 86


But let's just stick with a general $V$ for now. Call the number of possible states $S$. Let's consider the _transition matrix_ $T$ of this Markov chain. The transition matrix is a square matrix and the number of rows/columns is equal to the number of states, so in our case it is $S \times S$. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

**Your tasks:**

Answer the following questions:

1. What is the number of possible states, $S$, in terms of $V$ and $n$? (You might not encounter all these states in practice, if not all $n$-grams appear in the training data, but let's ignore that for now.)
2. Give two examples states in the state space for $n=3$.
3. In our application above when $n>1$ _not all transitions are possible_. This means $T$ will contain a bunch of zeros. Why?
4. Again for $n>1$ what is the maximum number of nonzero elements that $T$ could have? Answer in terms of $V$ and $n$. 
5. Is the state space discrete or continuous?

### YOUR ANSWER HERE

1. V^n

2. are, abl

3. When n = 3 for instance, and the current state is "are", the next state cannot be something like "bcd" because it should have atleast two characters in common with the previous state, since only the last character is being generated. The next state can be "re" followed by another character that is generated.

4. (V^n) * V

5. The state space is continuous because there are common characters between one state and the next.

<br><br><br><br>

### Exercise 3: Stationary distribution and other fun stuff <a name="3"></a>

The code below computes the transition matrix for you for the Shakespeare data with `n=1`. Consider this transition matrix for the next few questions.

In [109]:
# The Project Gutenberg eBook of The Complete Works of William Shakespeare, by William Shakespeare
data_url = "http://www.gutenberg.org/files/100/100-0.txt"
shakespeare_text = urlopen(data_url).read().decode("utf-8")
shakespeare_text = shakespeare_text[4000:]

In [110]:
states = np.unique(list(shakespeare_text))
print("States:", states)
S = len(np.unique(list(shakespeare_text)))
print("Number of states:", S)

model = MarkovModel(n=1)
model.fit(shakespeare_text)

# implementation note: since len(model.probabilities_[state]["probs"]) might not equal S
# for all letters, we need to be careful and do a reverse-lookup for the actual letters
# the rest of the transition probabilities are just zero (if they don't appear)
lookup = dict(zip(states, list(np.arange(S, dtype=int))))

T = np.zeros((S, S))  # transition matrix
for i, state in enumerate(states):
    for j, letter in enumerate(model.probabilities_[state]["symbols"]):
        T[i, lookup[letter]] = model.probabilities_[state]["probs"][j]

print("Number of nonzero transition probabilities: %d/%d" % (np.sum(T > 0), T.size))

States: ['\t' '\n' '\r' ' ' '!' '"' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.' '/'
 '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '?' '@' 'A' 'B' 'C' 'D'
 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V'
 'W' 'X' 'Y' 'Z' '[' '\\' ']' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '|'
 '}' 'Æ' 'É' 'à' 'â' 'æ' 'ç' 'è' 'é' 'ê' 'ë' 'î' 'œ' '—' '‘' '’' '“' '”']
Number of states: 107
107
Number of nonzero transition probabilities: 2647/11449


<br><br>

### 3.1 Stationary distribution conditions 
rubric={reasoning:5}

Under mild assumptions, a Markov chain has a _stationary distribution_ which is the probability of finding yourself in the same state after the chain is run for a long time. These assumptions are:

- "Irreducibility" (doesn’t get stuck in part of the graph)
- "Aperiodicity" (doesn’t keep repeating same sequence).

Give a short explanation for why our Markov chain might satisfy these assumptions.

### YOUR ANSWER HERE

### YOUR ANSWER HERE

Irreducibility: Every state(n-gram) that occurs is part of the frequencies dictionary. There is no state that is not part of the dictionary and hence every state has some predictions for the next character with certain probability. There is very low probability that two states will have only each other as their predictions. Hence, there is no probability of the graph getting stuck in some sequence.

Aperiodicity: Every state has the next character that might occur as a discrete probability set. It can be any character from that set. So, the process seems to be randomly picking a character from the values associated with the current state and performing thhis repeatedly. There is very low probability that a certain pattern might occur at **regular** intervals given the process is random.

<br><br>

### 3.2 Stationary distribution for the `shakespeare_text`
rubric={accuracy:10}

It's not true in general but in this particular case we actually already know the stationary distribution -- it is just the relative frequency that these n-grams occur in the training data (you are welcome to think about this deeply, but you don't have to). (Recall that we are assuming n=1 here.)

**Your tasks:** 

1. Calculate the stationary distribution for the `shakespeare_text` by calculating the relative frequencies for each state in the corpus (unique letter in this case, as we are assuming `n=1`).
2. Show empirically that this is a stationary distribution. 

In [133]:
### YOUR ANSWER HERE
### BEGIN SOLUTION

total_chars = len(shakespeare_text)
stationary_dist_freq = np.array(
    [shakespeare_text.count(state) / total_chars for state in states]
)
print(stationary_dist_freq)

# pi = pd.Series(lookup.values())/total_chars
# pi.index = lookup.keys()
# pi = np.array(pi)

# print(np.allclose(pi @ T, pi))

print(np.allclose(stationary_dist_freq @ T, stationary_dist_freq))

### END SOLUTION

[5.25125321e-07 2.98348201e-02 2.98348201e-02 1.84509258e-01
 1.49013062e-03 1.40033419e-05 3.50083547e-07 1.75041774e-07
 1.15527571e-05 2.49189469e-03 5.74137018e-05 5.72386600e-05
 4.55108612e-06 1.62132443e-02 1.01104129e-03 1.48939544e-02
 2.10050128e-06 2.87068509e-05 6.16147043e-05 3.97344826e-05
 2.87068509e-05 3.37830623e-05 2.03048458e-05 2.71314749e-05
 1.54036761e-05 2.59061825e-05 1.61038432e-05 8.24796838e-04
 2.99093879e-03 1.94313873e-03 1.75041774e-07 7.78095693e-03
 2.32087888e-03 3.28413376e-03 2.30232445e-03 6.31375678e-03
 1.98112279e-03 1.82165974e-03 2.94070180e-03 9.06016221e-03
 3.25052574e-04 1.00929087e-03 3.81328504e-03 2.53985614e-03
 4.35959042e-03 4.84515630e-03 1.84424013e-03 2.06199209e-04
 4.22288279e-03 5.34472552e-03 6.74610996e-03 2.24858663e-03
 5.58033175e-04 2.91129478e-03 6.42403310e-05 1.27045319e-03
 9.87235604e-05 6.09670498e-04 1.75041774e-07 6.09845540e-04
 9.38048865e-04 1.75041774e-07 4.64706152e-02 8.88109447e-03
 1.28183091e-02 2.549413

The equality statement evaluated to True. Hence, empirically proved

<br><br>

### (optional) 3.3 Stationary distribution using eigenvalue decomposition
rubric={reasoning:1}

You can compute the stationary distribution in another ways. 

**Your tasks**

1. Calculate stationary distribution using eigenvalue decomposition of the transition matrix. Compare your result from part 3.2 with the result below. Do they agree with each other?

> We haven't explicitly talked about this in class but at this point in the program, I believe that you are independent enough to do it on your own. 

In [112]:
### Solution_3_3_1
### YOUR ANSWER HERE

<br><br>

### 3.4 Finding probability of occurrences of patterns 
rubric={reasoning:15}

Let's consider the conditional probability that a lower case vowel comes 3 characters later given that the current letter is "a". In other words, we're searching for the pattern `axxv` where `v` is a lower case vowel (defined as a,e,i,o,u,y) and `x` is any character and `a` is literally "a". 

Let's use `n=1`.

**Your tasks:**
1. It turns out we can estimate this probability directly from the transition matrix. While $T$ gives us the probabilities one step ahead, $T\times T$ gives us the probabilities 2 steps ahead, etc. (If you want to think about this, you should be able to convince yourself it's true by contemplating what matrix multiplication really means, but this is optional.) So, taking $T^3$ gives us the transition probabilities for 3 steps later. Compute $T^3$ and find the estimated probability that we're looking for. 
2. We could also estimate this probability directly from the data, by just seeing how frequently this pattern occurs. Do this. How well does it match your above results? What does this tell us about how reasonable our  Markov assumption is?
3. What if we increased `n` and repeated part (1), would you expect to get closer to the answer from part (2)? You are welcome to try this but you don't have to - the goal is just to think about it.

> Note for 3.4.1: you should NOT use `T**3`, as that is element-wise exponentiation rather than a matrix power. You can get $T^3$ using `numpy.linalg.matrix_power(T,3)` or just `T@T@T`.

> This question might be a bit tricky. But hopefully it will help you understand Markov models better. 

In [113]:
### Solution_3_4_1

### YOUR ANSWER HERE

probs = np.linalg.matrix_power(T,3)
probs_df = pd.DataFrame(data = probs, index = states, columns = states)
probs_df.loc['a'].filter(items = ['a', 'e', 'i', 'o', 'u', 'y']).sum()

0.25512535358697536

In [114]:
### YOUR ANSWER HERE

In [115]:
### Solution_3_4_2

### YOUR ANSWER HERE
count = 0
context = 0
for i, char in enumerate(shakespeare_text):
    if char == 'a' and ((i + 3) < len(shakespeare_text)):
        count += 1
        if (shakespeare_text[i+3] in ['a', 'e', 'i', 'o', 'u', 'y']):
            context += 1
print(context/count)

0.2548411762711737


The obtained answer is very close to the one obtained from the transition matrix.

**Solution_3_4_3**

### YOUR ANSWER HERE

When we increase n, there are a lot more characters and the context is more prevalent than in the case with n = 1. Hence, it is easier to predict the next characters. Hence, I think the answers will be more closer. 

<br><br><br><br>

## Exercise 4: Markov model of language with words <a name="4"></a>

In this exercise we'll continue with the Markov model of language, but we'll work with _words_ instead of _characters_. The `MarkovModel` code stays the same. Just remember that now we are dealing with words and not characters. 

When we say n-gram we're now referring to `n` words, not `n` characters.

One concern with words is that, in a sense, we have less examples to work with. For example, with character n-grams and `n=3` we have a lot of examples of seeing the character combination "per" and checking what character follows it, but with words even with `n=1` (the smallest nontrivial value of `n`), we might only have one example of the word "persuade" in our corpus. You can imagine that the number of "training examples" only gets smaller if `n`>1. This is something to keep in mind when deciding what values of `n` might be reasonable.

Another issue is that the number of states could explode when you are working with words. For example, with character bigram model our states were all unique characters in the text (107 in our example above). For word bigram model, it's going to be the number of unique words in the corpus, which would be a much bigger number. You can imagine that for a large corpus and bigger values of `n`, the number of states could explode.  

Finally, we need to preprocess the text (i.e., tokenization) before creating a word-based n-gram model. To accomplish this preprocessing, we will use the popular python package [NLTK](http://www.nltk.org/) (Natural Language ToolKit), which we have seen in class. If you are using the course conda environment, you should already have the package in your conda environment. 

You might need to install the NLTK data files with the following command in your conda environment in the terminal.

```
python -m nltk.downloader 'punkt'
```

### 4.1 Word-based n-gram language model
rubric={accuracy:8,reasoning:4}

The first step is to break apart the text string into words. NLTK's `word_tokenize` will turn our text into a lists of "tokens" which we will use as our language units. This tokenizing process removes whitespace. So, in `generate`, you should add a space character back in between tokens. This won't look quite right because you'll have spaces before punctuations, but it's good enough for now.

**Your tasks:**

1. Implement a word-based Markov model of language.
2. Train your word-based Markov model on `shakespeare_text` above with your choice of `n` and generate some text. 
3. Discuss the differences between character-based and word-based Markov models in terms of generated text, the number of states, and the time taken to generate the text. 

> Note: You can reuse the code from Exercise 2. Only the `generate` function needs to be rewritten, and, even there, the required modifications are not large. You may use class inheritance you have learned in 511 here. The line
```
class MarkovModelWords(MarkovModel):
```
> defines a new class called `MarkovModelWords` that you can think of as an offspring of `MarkovModel`. It "inherits" all the functions from `MarkovModel` unless we explicitly overwrite them. So, if you implement `generate` in the class `MarkovModelWords`, the old `generate` function will be overwritten but all the other functions will persist. But, if you're not comfortable with inheritance, or just don't feel like it, you're  welcome to just modify the above code and not use inheritance.


> Another note: The way `MarkovModel` is implemented, the patterns we're conditioning on (the last $n$ characters/words) are used as keys in python dictionaries. For characters that was easy because we could just use a string as a key. However, for words with $n>1$ we now have a collection of words, and we want to use this as a dict key. `nltk.tokenize.word_tokenize` outputs a list, which can't be used as a key. I've casted the list to a tuple in the code below, because a tuple _can_ be used as a dict key (if you case: I believe this is because it's immutable whereas a list is not). This tiny change allows reuse of the old code.

In [116]:
import nltk
from nltk.tokenize import word_tokenize

text_tok = word_tokenize(shakespeare_text)
text_tok = tuple(text_tok)

In [121]:
### Solution_4_1_1

### YOUR ANSWER HERE
class MarkovModelWords:
    def __init__(self, n):
        """
        Initialize the Markov model object.

        Parameters:
        ----------
        n : int
            the size of the ngram
        """
        self.n = n

    def fit(self, text):
        """
        Fit a Markov chain and create a transition matrix.

        Parameters
        ----------
        text : str
            text to fit the Markov chain
        """

        # make text circular so markov chain doesn't get stuck
        circ_text = text + text[: self.n]
#         circ_text = text
        # count the number of occurrences of each letter following a given n-gram
        frequencies = defaultdict(Counter)
        for i in range(len(text)):
            frequencies[circ_text[i : i + self.n]][circ_text[i + self.n]] += 1.0

        # normalize the frequencies into probabilities (separately for each n-gram)
        self.probabilities_ = defaultdict(dict)
        for ngram, counts in frequencies.items():
            self.probabilities_[ngram]["symbols"] = list(counts.keys())
            probs = np.array(list(counts.values()))
            probs /= np.sum(probs)
            self.probabilities_[ngram]["probs"] = probs
        # store the first n characters of the training text, as we will use these
        # to seed our `generate` function
        self.starting_chars = text[: self.n]
        self.frequencies_ = frequencies  # you never know when this might come in handy

    def generate(self, seq_len):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
#         s = self.starting_chars
#         count = self.n
#         while count < seq_len:
#             result = s[-self.n:]
#             word_gram = " ".join(result)
#             probs = self.probabilities_[word_gram]
#             print(probs)
#             s += list(npr.choice(probs["symbols"], p=probs["probs"]))
#             count += 1
#         return s
        count = self.n
        s = list(self.starting_chars)
        while count < seq_len:
            probs = self.probabilities_[tuple(s[-self.n :])]
#             print(f"""n-gram : {tuple(s[-self.n :])} corresponding value : {probs}""")
            pred = npr.choice(probs["symbols"], p=probs["probs"])
            s += [pred]
            count += 1
        s = " ".join(s)
        return s

In [122]:
### Solution_4_1_2
### YOUR ANSWER HERE
model = MarkovModelWords(n=3)
model.fit(text_tok)
result = model.generate(50)
result

'serv ’ d him at the barber ’ s shop is hanging still , That mouldeth goblins swift as frenzy ’ s thoughts . Strike a free march to Troy . With comfort go ; Hope of revenge shall hide our inward woe . Enter Pandarus . CRESSIDA . A'

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

**Solution_4_1_3**

### YOUR ANSWER HERE

When I printed the length of the frequencies dictionary for word based model, the length is much higher than that of character based. I think this is because a lot more combinations are possible in word based model as compared to character based model. The time taken to fit is lesser than the time taken for charcter based model. The generate function speed is not very different for both the models. 

<br><br>

### 4.2 Dealing with punctuation
rubric={accuracy:10,reasoning:4}

If you've generated text from 4.1, you probably noticed that the way whitespace is handled is not ideal. In particular, whitespace is added after every token, including right before punctuation. 

**Your tasks:**

1. Modify your code to fix this and show a generated sequence. You can access a list of punctuation characters with `string.punctuation` from the [`string` library](https://docs.python.org/3/library/string.html).
2. Generate text using different values of `n` on `shakespeare_text`.  
3. Train word-based Markov models on another corpus of your choice and generate text with different values of `n`. You may use any corpus of your choice. For inspiration, you may find some corpora [here](http://www.gutenberg.org/). 
4. Note your observations on 
    - how the generated text changes based on the corpus 
    - how the value of `n` affects the quality of the generated text and time taken to generate the text

In [102]:
### Solution_4_2_1
### YOUR ANSWER HERE

### YOUR ANSWER HERE
class MarkovModelWordsPunc:
    def __init__(self, n):
        """
        Initialize the Markov model object.

        Parameters:
        ----------
        n : int
            the size of the ngram
        """
        self.n = n

    def fit(self, text):
        """
        Fit a Markov chain and create a transition matrix.

        Parameters
        ----------
        text : str
            text to fit the Markov chain
        """

        # make text circular so markov chain doesn't get stuck
        circ_text = text + text[: self.n]
#         circ_text = text
        # count the number of occurrences of each letter following a given n-gram
        frequencies = defaultdict(Counter)
        for i in range(len(text)):
            frequencies[circ_text[i : i + self.n]][circ_text[i + self.n]] += 1.0

        # normalize the frequencies into probabilities (separately for each n-gram)
        self.probabilities_ = defaultdict(dict)
        for ngram, counts in frequencies.items():
            self.probabilities_[ngram]["symbols"] = list(counts.keys())
            probs = np.array(list(counts.values()))
            probs /= np.sum(probs)
            self.probabilities_[ngram]["probs"] = probs
        # store the first n characters of the training text, as we will use these
        # to seed our `generate` function
        self.starting_chars = text[: self.n]
        self.frequencies_ = frequencies  # you never know when this might come in handy

    def generate(self, seq_len):
        """
        Using self.starting_chars, generate a sequence of length seq_len
        using the transition matrix created in the fit method.

        Parameters
        ----------
        seq_len : int
            the desired length of the sequence

        Returns:
        ----------
        str
            the generated sequence
        """
        count = self.n
        s = list(self.starting_chars)
        while count < seq_len:
            probs = self.probabilities_[tuple(s[-self.n :])]
            pred = npr.choice(probs["symbols"], p=probs["probs"])
            s += [pred]
            count += 1
        s = " ".join(s)
        expr = f'''\s+(?=[{string.punctuation}'])'''
        s = re.sub(expr, '', s)
        expr2 = f'''\'\s+'''
        s = re.sub(expr2, '\'', s)
        return s
# class MarkovModelWords(MarkovModel):
#     def generate(self, seq_len):
#         n = self.n
#         seq = self.starting_chars
#         while len(seq) < seq_len:
#             probs = self.probabilities_[seq[-n:]]
#             seq += (npr.choice(probs["symbols"], p=probs["probs"]),)
#         output = seq[0]
#         for s in seq[1:]:
#             if s[0] in string.punctuation:
#                 output += s
#             else:
#                 output += " " + s
#         return output

In [103]:
### Solution_4_2_2
### YOUR ANSWER HERE
model = MarkovModelWordsPunc(n=3)
model.fit(text_tok)
result = model.generate(50)
result

'All were surprised by Merriwell ’ s final words were a thrust at him. The owner of those eyes drew back in a moment. Ha! I thought as much! I saw you down at the wharf, Dustan met him at the head of the stairs'

In [104]:
### Solution_4_2_3
### YOUR ANSWER HERE
import urllib.request
sample_data_url = 'http://www.gutenberg.org/files/63483/63483-0.txt'
sample_corpus = urllib.request.urlopen(sample_data_url).read().decode("utf-8")
sample_corpus = sample_corpus[9300:34900]
text_tok = word_tokenize(sample_corpus)
text_tok = tuple(text_tok)
sample_model = MarkovModelWordsPunc(n = 5)
sample_model.fit(text_tok)
sample_result = sample_model.generate(500)
print(sample_result)

All were surprised by Merriwell ’ s sudden move. Frank had seen a person appear in the open door of the freight house, look at him, and then dodge back. Although he obtained but a glimpse of this person, Merry fancied he knew him. Into the doorway he sprang, and looked around. On every hand were boxes and barrels and piles of freight, but no one was to be seen. The opposite door was standing open. “ Must have dodged out that way, ” muttered Frank, and he darted toward the door. But when he reached the door, he looked in vain for the person he fancied he had seen. “ My eyes may have fooled me, ” he said. “ We have a hard hill out here, ” said Woodock. “ Some of us can ’ t climb it, ” smiled Frank. “ I think there is no danger of your failing. ” They started. From choice it seemed Mabel Mischief rode at Frank ’ s side, chatting with Hattie Hazle, who was next to her. Hattie had dark eyes and hair, presenting a strong contrast to the lively blonde. The hill proved to be rather steep and d

**Solution_4_2_4**
### YOUR ANSWER HERE

As n increases, the quality of text generated, significantly increases. Also, with classic english literature, the generated text seems to be of not very high quality. With other corpus, the text generated seems to be better.

<br><br>

### (optional) 4.3 Other improvements
rubric={reasoning:1}

Are there other improvements you'd like to make to your word-based Markov model so that the generated text looks like proper English? Feel free to either implement them or just describe what you might want to do.

**Solution_4_3**
### YOUR ANSWER HERE

<br><br><br><br>

## Submission to Canvas <a name="sc"></a>

**PLEASE READ: When you are ready to submit your assignment do the following:**

- Run all cells in your notebook to make sure there are no errors by doing Kernel -->  Restart Kernel and Run All Cells...
- If you are using the "575" `conda` environment, make sure to select it before running all cells. 
- Convert your notebook to .html format using the `convert_notebook()` function below or by File -> Export Notebook As... -> Export Notebook to HTML
- Run the code `submit()` below to go through an interactive submission process to Canvas.
After submission, be sure to do a final push of all your work to GitHub (including the rendered html file).

In [ ]:
# from canvasutils.submit import convert_notebook, submit

# convert_notebook("lab1.ipynb", "html")  # uncomment and run when you want to try convert your notebook (or you can convert manually from the File menu)
# submit(course_code=65512, token=False)  # uncomment and run when ready to submit to Canvas

Well done!! Congratulations on finishing the lab!! 